# Running a job

Below are example submission scripts for several different codes.

These examples copy the run data from the submission directory in `/campaign/` area to `/scratch/` before the run, and copy any files back to `/campaign/` afterwards. The script then cleans up by removing the `working_directory` in the `/scratch/` area.


Obviously there are an infinite number of ways to structure the logic in your submission scripts in order to handle the data transfer between `/campaign/` and `/scratch/`.

The following examples can be adapted to your own needs. They should be submitted from the `/campaign/` storage area, they will create a temporary working directory `/scratch/$USER/$SLURM_JOB_ID` and copy the contents of the submission folder over to the working directory. After the run it copies the contents of the working directory in `scratch` back to the submission directory in `/campaign/`. It then cleans up the scratch working directory.

## Openfoam

We will run an example job using the OpenFoam 7 module for 

`OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/v7`

Let's load the module:
```bash
module load OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/v7
```

Source the OpenFoam bashrc:

```bash
source $foamDotFile```

Lets switch to our scratch space:

```bash
cd $SCRATCH
```


And copy over the run files:
``` bash
cp -r $WM_PROJECT_DIR/tutorials/multiphase/interFoam/laminar/damBreak ./```

An example run script:
    
```bash
#!/bin/bash
#SBATCH --account=prj3_phase1
#SBATCH --job-name=JOB_NAME
#SBATCH --output=JOB_NAME.%j.o
#SBATCH --error=JOB_NAME.%j.e
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=120
#SBATCH --partition=spot-hbv3-120
#SBATCH --qos=spot-hbv3-120
#SBATCH --time=04:00:00

module purge
module load OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/v7

# As an example of moving input data from /campaign
# to scratch we will create and mirror the submit
# directory structure on scratch:

campaigndir=$(pwd)
workdir=$SCRATCH/$SLURM_JOB_ID
mkdir -p $workdir

echo $campaigndir | if grep -q campaign; then 
    # Copy any inputs required to the work directory:
    # excluding and files with a JOB_NAME prefix, so your
    # output and error files don't get over written on the copy back
    rsync -aP --exclude=JOB_NAME.* $campaigndir/* $workdir/
    cd $workdir;
    echo  "Work directory" $workdir ;
    
    # source the foamDotFile and do the run
    . $foamDotFile
    ./Allrun

    # Copy back results.
    srun cp -Rf $workdir/*  $campaigndir/
    #Clean up scratch
    rm -rf $workdir
else
     echo "Not submitting from campaign. Check your submission script."
     exit 1;
fi


```

Lets switch into the folder 
```bash
cd damBreak
```

And after copying the above runscript into `run_job.slm` lets submit the job:

```bash
sbatch run_job.slm
```

## ANSYS

Below is an example script for an Ansys run, which should be submitted from the `/campaign/` storage area.

```bash
#!/bin/bash
#SBATCH --job-name=JOB_NAME
#SBATCH --output=JOB_NAME.%j.o
#SBATCH --error=JOB_NAME.%j.e
#SBATCH --account=sysadmin
#SBATCH --partition=spot-fsv2-32
#SBATCH --qos=spot-fsv2-32
#SBATCH --time=2:00:00
#SBATCH --nodes=1
#SBATCH --tasks-per-node=32


env | grep SLURM | sort -d
echo

export CASE="Free_Surf"

## Load the required modules
module load ansys/v212
module list

# As an example of moving input data from /campaign
# to scratch we will create and mirror the submit
# directory structure on scratch:

campaigndir=$(pwd)
workdir=$SCRATCH/$SLURM_JOB_ID
mkdir -p $workdir

echo $campaigndir | if grep -q campaign; then 
    # Copy any inputs required to the work directory:
    # excluding and files with a JOB_NAME prefix, so your
    # output and error files don't get over written on the copy back
    rsync -aP --exclude=JOB_NAME.* $campaigndir/* $workdir/
    echo  "Work directory" $workdir ;
    ## Define files
    export DEFFILE="${CASE}.def"

    ## Define additional options
    OPTIONS="-double -batch"

    CFX_PARALLEL="$CFX_BATCH_MODE $CFX_CCL_FILE $CFX_START_METHOD $CFX_PARTITION $CFX_PARTITIONER"
    
    HOSTS=$(srun --ntasks-per-node $SLURM_CPUS_ON_NODE hostname | sort | uniq -c | awk '{print $2 "*" $1}' | paste -sd, -)
    echo "HOSTS: $HOSTS"
    echo ""

    # Run CFX5SOLVE
        cfx5solve -def Free_Surf.def ${OPTIONS} -start-method 'Intel MPI Distributed Parallel' -par-dist $HOSTS -partitioner ${CFXPARTITIONER}

    # Copy back results.
    srun cp -Rf $workdir/*  $campaigndir/
    #Clean up scratch
    rm -rf $workdir
else
    echo "Not submitting from campaign. Check your submission script."
    exit 1;
fi

```

### Matlab

A hello world matlab script 'hello_world.m':

```matlab
disp('Hello, World');
```

An example run-script for a matlab job, which 
can be submitted from /campaign:

```bash
#!/bin/bash
# Account to charge to
#SBATCH --account=prj4_phase1
#Name of the job
#SBATCH --job-name=JOB_NAME
#SBATCH --output=JOB_NAME.%j.o
#SBATCH --error=JOB_NAME.%j.e
# Number of nodes requested
#SBATCH --nodes=1
# The no of tasks per node
#SBATCH --ntasks-per-node=32
# The partition to submit to
#SBATCH --partition=spot-fsv2-32
# The quality of service
#SBATCH --qos=spot-fsv2-32
# The runtime
#SBATCH --time=04:00:00


# Purge modules and load matlab
module purge
module load matlab/2020b

# As an example of moving input data from /campaign
# to scratch we will create and mirror the submit
# directory structure on scratch:

campaigndir=$(pwd)
workdir=$SCRATCH/$SLURM_JOB_ID
mkdir -p $workdir

echo $campaigndir | if grep -q campaign; then 
    # Copy any inputs required to the work directory:
    # excluding and files with a JOB_NAME prefix, so your
    # output and error files don't get over written on the copy back
    rsync -aP --exclude=JOB_NAME.* $campaigndir/* $workdir/
    echo  "Work directory" $workdir ;
    matlab -nosplash -nodisplay -r "hello_world; exit"

    # Copy back results.
    srun cp -Rf $workdir/*  $campaigndir/
    #Clean up scratch
    rm -rf $workdir
else
    echo "Not submitting from campaign. Check your submission script."
    exit 1;
fi




```

### Interactive job

It is also possible to obtain a compute node and work interactively. 

Issuing the following command in the terminal will request a spot-hbv3-120 compute instance for 6 hours:

```
srun --partition spot-hbv3-120 --nodes 1  --account prj4_phase1  --qos prj4_phase1 --job-name "interactive" --cpus-per-task 120 --time 6:00:00 --pty bash
```